In [1]:
# Import necessary libraries
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.models as models
from torch.utils.data import DataLoader


In [2]:
# Initialize and load the ResNet-50 model with pre-trained weights
model = models.resnet50(weights=models.ResNet50_Weights.IMAGENET1K_V1)
model.eval()  # Set the model to evaluation mode


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [3]:
# Define the transformations for the dataset
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [4]:
# Function to download and prepare the dataset
from utility.get_data import get_data

# Set the parameters for data loading
batch_size = 32
img_root = "imagenet-a"  # Ensure this matches the root folder in the S3 bucket
seed = 42

# Get data loaders
train_loader, val_loader, test_loader = get_data(batch_size, img_root, seed, transform)

# Combine loaders into a dictionary for easy access
data_loaders = {
    'train_loader': train_loader,
    'val_loader': val_loader,
    'test_loader': test_loader
}


In [6]:
# Define the Tester class to test the model
from train_test.test import Tester

# Set the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Initialize the loss function
loss_fn = nn.CrossEntropyLoss()

# Initialize the Tester class
tester = Tester(model, data_loaders, device, loss_fn, num_classes=1000)

In [7]:
# Test the ResNet-50 model on the ImageNet-A dataset
test_loss, test_accuracy = tester.test_step(test=True)

# Print the test results
print(f'Test Loss: {test_loss:.4f}')
print(f'Test Accuracy: {test_accuracy:.2f}%')

Test Loss: 0.3924
Test Accuracy: 0.00%
